In [ ]:
pip install xlsxwriter

In [ ]:
pip install openpyxl

In [ ]:
pip install -U sentence-transformers


In [ ]:
import sys
sys.path.append('../input')
from xuexibao.utils.fileUtils import *
from sentence_transformers import SentenceTransformer, util


In [ ]:
lines = readExcelToList('../input/fc4yue4hao/ground_truth-500.xlsx')

In [ ]:
data = lines[1:501]

In [ ]:
data[0]

In [ ]:
lines[0]

In [ ]:
lines[1]

In [ ]:
data1 = [i[10] for i in data]
data2 = [i[2] for i in data]

In [ ]:
data1[-1],data2[-1]

In [ ]:
f1 = readExcelToList('../input/facebook/f1.xlsx')
def ty_comment(name):
    s_comment = ''
    for i,j in f1:
        if j == name:
            s_comment+=i
    return s_comment

In [ ]:
comment = ty_comment('Disclosure')+ty_comment('Usage')+ty_comment('Collection')+ty_comment('Retention')

In [ ]:
# sentence-transformers/multi-qa-MiniLM-L6-cos-v1
# sentence-transformers/paraphrase-MiniLM-L6-v2 
# sentence-transformers/all-MiniLM-L6-v2
# sentence-transformers/multi-qa-mpnet-base-dot-v1
# sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2 
# sentence-transformers/all-mpnet-base-v2

In [ ]:
model_name = ['sentence-transformers/paraphrase-MiniLM-L6-v2','sentence-transformers/all-MiniLM-L6-v2','sentence-transformers/multi-qa-mpnet-base-dot-v1','sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2','sentence-transformers/all-mpnet-base-v2','sentence-transformers/LaBSE']

In [ ]:
mkdir z

In [ ]:
def find_coment(query,docs,path,model,data):
    #Encode query and documents
    query_emb = model.encode(query)
    doc_emb = model.encode(docs)

    #Compute dot score between query and all document embeddings
    scores = util.pytorch_cos_sim(query_emb, doc_emb).cpu().tolist()
    print(len(scores[0]))
    #Combine docs & scores
    doc_score_pairs = list(zip(docs, scores[0],data))

    #Sort by decreasing score
    doc_score_pairs = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)
    #print(doc_score_pairs[0])
    f = open(path,'w')
    f.write('id,socre,doc,result,Precision,,AP\n')
    count = 0
    flag = True
    for doc, score,data in doc_score_pairs:
        count+=1
        if flag:
            f.write('{},{},"{}",{},"=COUNTIF($D$2:D2,1)/COUNT($D$2:D2)","=IF(D2=1,E2,0)","=SUM($F$2:F2)/A2"\n'.format(count,score,doc,data))
            flag = False
        else:
            f.write('{},{},"{}",{}\n'.format(count,score,doc,data))

In [ ]:
for i in model_name:
    find_coment(comment,data1,'./z/{}.csv'.format(i.split('/')[-1]),SentenceTransformer(i),data2)